# IMPORTANT
I strongly recommend first checking out the other document, it's the first part of my Open Program. This docuemnt is more of a part 2. You can still read through it but i have put lots of effort in the other document aswel as this document.

# Table of Contents
- [Introduction](#Introduction)
- [Train Model (5 Frames)](#Train-Model-(5-frames))
- [Test Model (5 Frames)](#Test-Model-(5-Frames))
- [Train Model (2 Frames)](#Train-Model-(2-frames))
- [Test Model (2 Frames)](#Test-Model-(2-Frames))
- [Troubleshooting](#Troubleshooting)
- [My Model vs Random Agent (Rematch)](#My-Model-vs-Random-Agent-(Rematch))
- [Conclusion](#Conclusion)

<h1 style="background-color: #9cffb6; text-align:center">Introduction</h1>

You made it! this is the third and last document of my Open Program. in the last document (cart-pole-game-v2) we tried creating a model using rewards based of values in game, like pole speed and pole angle. Today, You will read how I attempt to create a model that learns by only uses being alive as reward. We will try to look steps into the future whilst playing (and look steps back while training). I made this into a new document since the last one was super messy where i declared the same function over 6 times and it was made in the middle of the night so screw that. Time for the REAL work. 

**Goal:** hit a score of 500. 1 score point is equal to 1 being alive for one frame

## What have we learned last time
recognizing game states works on both 1 decimal and 2 decimals. However, 2 decimals takes really long to train and runs slower in the end, plus 1 decimals is enough for good results. My previous document can vouch that
- In short, use 1 decimal to save states

## Let's get started 😎


## Install dependencies

In [2]:
!pip install gym
!pip install gym[box2d]

## Import Dependencies

In [3]:
import gym
print("gym version: " + gym.__version__)
import random

gym version: 0.26.2


## Coding

### How much frames should we look ahead?
This is the first question that came in mind. We could obviously try out different amount of frames, but to save time we will make an agent hold one direction the entire time and see how long it takes to game over. We will take this amount and substract a few frames of it lets say 5. We do this because usually one frame before you die you cant recover.

So let's start making an agent that just holds left

In [15]:
def hold_left(MAX_AMOUNT_OF_TRIES, MAX_AMOUNT_OF_FRAMES_PER_TRY):
    # a try at the game
    for episode in range(MAX_AMOUNT_OF_TRIES):
        # reset all previous made progress in the environment
        env.reset()
        score=0
        for t in range(MAX_AMOUNT_OF_FRAMES_PER_TRY):
            # make the environment visible
            env.render()
            
            # generate random action (0=left or 1=right)
            #action = env.action_space.sample()
            action = 0

            # output of the action
            _, _, done, _, _ = env.step(action)
            
            # print results
            score+=1
            if done:
                print('score: ' + str(score))
                break

Now let's test it and see how much frames it takes to game over

In [18]:
#env = gym.make("CartPole-v1")
env = gym.make("CartPole-v1", render_mode="human") # uncomment this to actually see the progress (slows down the process terribly)

MAX_AMOUNT_OF_TRIES = 5
MAX_AMOUNT_OF_FRAMES_PER_TRY = 500

hold_left(MAX_AMOUNT_OF_TRIES, MAX_AMOUNT_OF_FRAMES_PER_TRY)

score: 10
score: 11
score: 10
score: 10
score: 9


We get an average score of 10, so let's try to train a model that can look 5 frames into the future whilst playing.

# How in the world are you planning to do this???
So whilst playing around in my previous notebook (v2), I saw an opportunity to look into the future whilst playing. The way I could to this goes as follows.

Let's say a new game starts. The first frame we do a move and save it. 5 frames later, we check if we are still alive. If so, the move done 5 frames ago was a good move. The rest is just repetition from v2 notebook. We save a state together with the action. Regarding rewards, we will give the action done 5 frames ago a fixed reward of 0, if it survives we can change this reward to 1. if it doesnt survive, last 5 moves will all get a reward of 0. Hope this sounds logically, let's get started.

Let's start with writing code that can remember last 5 actions.

In [140]:
def add_action(action_history, action, max_actions_saved):
    action_history.append(action)
    if len(action_history) > max_actions_saved:
        action_history.pop(0)
    print('action history')
    print(action_history)
    print('')

let's do the same thing for states

In [141]:
# I know i can just use the same function twice but it's nice to have 2 functions with different names for different yk stop hating pls 😿
def add_state(state_history, state, max_states_saved):
    state_history.append(state)
    if len(state_history) > max_states_saved:
        state_history.pop(0)
    print('state history:')
    print(state_history)
    print('')

a train function

In [112]:
REMEMBER_AMOUNT_OF_FRAMES = 5

# don't be scared about the new parameters, it's just better fitting names.
def train_1(attempts, max_score):
    
    # empty move history each game
    state_history = []
    action_history = []
    
    for episode in range(attempts):
        # reset all previous made progress in the environment
        env.reset()
        score=0
        for frame in range(max_score):
            # make the environment visible
            env.render()
            
            # generate random action (0=left or 1=right)
            action = env.action_space.sample()
            print('current frame: ' + str(frame))
            print('')
            add_state(state_history, env.state, REMEMBER_AMOUNT_OF_FRAMES)
            add_action(action_history, action, REMEMBER_AMOUNT_OF_FRAMES)
            print('--------------------')
            
            # output of the action
            _, _, done, _, _ = env.step(action)
            
            # print results
            score+=1
            if done:
                print('score: ' + str(score))
                print('')
                print('last 5 action were bad')
                print(action_history)
                break

If you look below you can see that the last 5 actions and states are being remembered.

In [113]:
train_1(1, 500)

current frame: 0

state history:
[array([ 0.00185766,  0.03647561,  0.02603861, -0.02709232])]

action history
[0]

--------------------
current frame: 1

state history:
[array([ 0.00185766,  0.03647561,  0.02603861, -0.02709232]), (0.0025871692856289695, -0.15900988919077705, 0.02549676089888423, 0.2736910196082215)]

action history
[0, 0]

--------------------
current frame: 2

state history:
[array([ 0.00185766,  0.03647561,  0.02603861, -0.02709232]), (0.0025871692856289695, -0.15900988919077705, 0.02549676089888423, 0.2736910196082215), (-0.0005930284981865716, -0.3544861834833104, 0.03097058129104866, 0.5743053946347696)]

action history
[0, 0, 0]

--------------------
current frame: 3

state history:
[array([ 0.00185766,  0.03647561,  0.02603861, -0.02709232]), (0.0025871692856289695, -0.15900988919077705, 0.02549676089888423, 0.2736910196082215), (-0.0005930284981865716, -0.3544861834833104, 0.03097058129104866, 0.5743053946347696), (-0.00768275216785278, -0.5500283305289201, 0

now, the 6th frame we want to be able to tell if the action performed on the first frame was a success. and for the 7th frame we want to be able to tell if the action performed on the 2nd frame was good etcetera

In [114]:
def observe_action(frame_max, current_frame, game_over, action_history):
    if game_over==False:
        frame_observe = current_frame - frame_max
        print('action performed on frame ' + str(frame_observe + 1) + ': ' + str(action_history[0]))
    

here we implement `observe_action`

In [115]:
REMEMBER_AMOUNT_OF_MOVES = 5

def train_2(attempts, max_score):
    action_history = []
    for episode in range(attempts):
        # reset all previous made progress in the environment
        env.reset()
        score=0
        for frame in range(max_score):
            print('current frame: ' + str(frame))
            # make the environment visible
            env.render()
            
            # generate random action (0=left or 1=right)
            action = env.action_space.sample()
            add_move(action_history, action, REMEMBER_AMOUNT_OF_MOVES)
            
            # output of the action
            _, _, done, _, _ = env.step(action)
            
            # check if move 5 frames ago was good
            if frame >= REMEMBER_AMOUNT_OF_MOVES - 1:
                observe_action(REMEMBER_AMOUNT_OF_MOVES, frame, done, action_history)
            
            # print results
            score+=1
            if done:
                break
        print('score: ' + str(score))

So it might be a bit hard to see but we have exactly what I want. on frame 4 we can look back to frame 0 and see what move was performed there.  (we can of course do this for states aswel but just for showcasing purposes i chose actions only)

In [82]:
train_2(1, 10)

current frame: 0
[1]
current frame: 1
[1, 0]
current frame: 2
[1, 0, 1]
current frame: 3
[1, 0, 1, 0]
current frame: 4
[1, 0, 1, 0, 0]
move performed on frame 0: 1
current frame: 5
[0, 1, 0, 0, 1]
move performed on frame 1: 0
current frame: 6
[1, 0, 0, 1, 1]
move performed on frame 2: 1
current frame: 7
[0, 0, 1, 1, 0]
move performed on frame 3: 0
current frame: 8
[0, 1, 1, 0, 1]
move performed on frame 4: 0
current frame: 9
[1, 1, 0, 1, 1]
move performed on frame 5: 1
score: 10


now that we can do this, we will actually start training. We will do this the same way as we did in the v2 notebook. We will save entries in a long array like the following. [state, action, reward]. Important to mention is that we not only save the action from 5 frames ago, we also save the state from 5 frames ago.

### Are you still confused?
You might be thinking, _"But now you collect data for past actions, actions that have been previously performed. How will you predict what you should do?"_ Well, when we will test our model, we can use the data we collected by looking back 5 frames each frame. if we use this data while testing, we can use it to look 5 frames into the future. If you are still confused, don't worry I can't explain it better 😿

## Fuction to collect data
You might recognize the following function. With this function we can collect data

In [122]:
DECIMALS = 1

def add_brain_cell(state, action, reward):
    # declare i so we can keep track of the current item in the loop and modify it
    i = 0
    for entry in brain_array:
        # for some reason the state is not an array so we make it an array first
        state_but_array = [round(state[0], DECIMALS), round(state[1], DECIMALS), round(state[2], DECIMALS), round(state[3], DECIMALS)]
        if entry[0] == state_but_array:
            #print('same state')
            if reward > entry[2]:
                print('better reward')
                brain_array[i] = [state_but_array, action, reward]
                return True 
            else:
                break
        i+=1
    brain_array.append([state_but_array, action, reward])

## Function to calculate reward
the reward function is way less complex this time. Now we just check if we are dead and that's it really

In [123]:
def calc_reward(game_over):
    
    # declare reward - returns 0 if dead
    reward = 0
    
    if game_over == False:
        reward = 1
    return reward

## Time for action
So, now we will save last 5 states, last 5 actions, calculate a reward and save all values all together.

In [129]:
# clear brain array first - and fill it with one stupid data entry (if its empty it crashes, we can if-statement to prevent crash but that would slow down unnecisarily)
brain_array = [[[0.00680, 0.00819, 0.02738, -0.01811], 0, 0]]

In [130]:
REMEMBER_AMOUNT_OF_FRAMES = 5

def train(attempts, max_score):
    
    # empty history each game
    state_history = []
    action_history = []
    
    for episode in range(attempts):
        
        # reset all previous made progress in the environment
        env.reset()
        score=0
        
        for frame in range(max_score):
            
            # make the environment visible
            env.render()
            
            # generate random action (0=left or 1=right)
            action = env.action_space.sample()
            
            # output of the action
            _, _, game_over, _, _ = env.step(action)
            
            # the good stuff
            add_state(state_history, env.state, REMEMBER_AMOUNT_OF_FRAMES)
            add_action(action_history, action, REMEMBER_AMOUNT_OF_FRAMES)
            reward = calc_reward(game_over)
            add_brain_cell(state_history[0], action_history[0], reward)
            
            # print results
            score+=1
            if game_over:
                break
        print('score: ' + str(score))

In [131]:
train(1, 10)

state history:
[(-0.022977336273136178, -0.21393823057464295, -0.006608882029072068, 0.3212037985387597)]

action history
[0]

state history:
[(-0.022977336273136178, -0.21393823057464295, -0.006608882029072068, 0.3212037985387597), (-0.027256100884629035, -0.40896544545087876, -0.00018480605829687344, 0.6117952350042442)]

action history
[0, 0]

state history:
[(-0.022977336273136178, -0.21393823057464295, -0.006608882029072068, 0.3212037985387597), (-0.027256100884629035, -0.40896544545087876, -0.00018480605829687344, 0.6117952350042442), (-0.03543540979364661, -0.21384091181429815, 0.01205109864178801, 0.31905410656665023)]

action history
[0, 0, 1]

state history:
[(-0.022977336273136178, -0.21393823057464295, -0.006608882029072068, 0.3212037985387597), (-0.027256100884629035, -0.40896544545087876, -0.00018480605829687344, 0.6117952350042442), (-0.03543540979364661, -0.21384091181429815, 0.01205109864178801, 0.31905410656665023), (-0.03971222802993257, -0.40913240472635165, 0.01843

Since we are doing complex stuff, I want to make sure everything is going correctly so let's do a quick check ourselves.

In [133]:
for entry in brain_array:
    print(entry)

[[0.0068, 0.00819, 0.02738, -0.01811], 0, 0]
[[-0.0, -0.2, -0.0, 0.3], 0, 1]
[[-0.0, -0.2, -0.0, 0.3], 0, 1]
[[-0.0, -0.2, -0.0, 0.3], 0, 1]
[[-0.0, -0.2, -0.0, 0.3], 0, 1]
[[-0.0, -0.2, -0.0, 0.3], 0, 1]
[[-0.0, -0.4, -0.0, 0.6], 0, 1]
[[-0.0, -0.2, 0.0, 0.3], 1, 1]
[[-0.0, -0.4, 0.0, 0.6], 0, 1]
[[-0.0, -0.6, 0.0, 0.9], 0, 1]
[[-0.1, -0.4, 0.0, 0.6], 1, 1]


WO WO WOW OW ok, so so so, hear me out. This is amazing. Without realising i already left out the bad moves. (moves with reward of 0). This means multiple things.
- Our model doesn't have to overwrite old data, this means
- - Training will be quicker
- - the model can play the game smoother since the proces is quicker

so overall amazing.

## Oppurtunity to speed up the process 
Since reward is always 1, we can completely leave it out from brain_array, making processes quicker.

In [154]:
DECIMALS = 1

def add_brain_cell_(state, action):
    state_but_array = [round(state[0], DECIMALS), round(state[1], DECIMALS), round(state[2], DECIMALS), round(state[3], DECIMALS)]
    brain_array.append([state_but_array, action])

comment out `print()` from both functions here for more speed

In [143]:
def add_action(action_history, action, max_actions_saved):
    action_history.append(action)
    if len(action_history) > max_actions_saved:
        action_history.pop(0)
    #print('action history')
    #print(action_history)
    #print('')

In [144]:
def add_state(state_history, state, max_states_saved):
    state_history.append(state)
    if len(state_history) > max_states_saved:
        state_history.pop(0)
    #print('state history:')
    #print(state_history)
    #print('')

Let's modify our `train()` function so that we can use it for quicker training

In [172]:
REMEMBER_AMOUNT_OF_FRAMES = 5

def train_model(attempts, max_score, visuals='off'):
    
    if visuals == 'off':
        env = gym.make("CartPole-v1")
    else:
        env = gym.make("CartPole-v1", render_mode="human") # uncomment this to actually see the progress (slows down the process terribly)

    
    # empty history each game
    state_history = []
    action_history = []
    
    for episode in range(attempts):
        
        # reset all previous made progress in the environment
        env.reset()
        score=0
        
        for frame in range(max_score):
            
            # make the environment visible
            env.render()
            
            # generate random action (0=left or 1=right)
            action = env.action_space.sample()
            
            # output of the action
            _, _, game_over, _, _ = env.step(action)
            
            # the good stuff
            add_state(state_history, env.state, REMEMBER_AMOUNT_OF_FRAMES)
            add_action(action_history, action, REMEMBER_AMOUNT_OF_FRAMES)
            add_brain_cell_(state_history[0], action_history[0])
            
            # print results
            score+=1
            if game_over:
                break
        print('score: ' + str(score))

<h1 style="background-color: #9cffb6; text-align:center">Train Model (5 frames)</h1>

first lets declare functions to save and load models

In [145]:
import json

def load_model(model):
    with open(model + '.json', 'r') as f:
        data = json.load(f)
        return data
    
def save_model(model, data):
    save_file = open(model + '.json', "w")  
    json.dump(data, save_file, indent = 6)  
    save_file.close() 

A function to get the average score

In [202]:
def get_avg_score(score_array, max_games):
    total = 0
    for x in score_array:
        total+=x
    return total/max_games

now we train

In [169]:
train_model(5, 500)

score: 22
score: 16
score: 14
score: 43
score: 30


In [170]:
brain_array

[[[0.0, 0.0, 0.0, 0.0], 0],
 [[0.0, -0.2, 0.0, 0.3], 0],
 [[0.0, -0.2, 0.0, 0.3], 0],
 [[0.0, -0.2, 0.0, 0.3], 0],
 [[0.0, -0.2, 0.0, 0.3], 0],
 [[0.0, -0.2, 0.0, 0.3], 0],
 [[0.0, -0.4, 0.0, 0.6], 0],
 [[0.0, -0.6, 0.0, 0.9], 0],
 [[0.0, -0.4, 0.1, 0.6], 1],
 [[0.0, -0.6, 0.1, 0.9], 0],
 [[0.0, -0.4, 0.1, 0.6], 1],
 [[-0.0, -0.6, 0.1, 1.0], 0],
 [[-0.0, -0.8, 0.1, 1.3], 0],
 [[-0.0, -0.6, 0.1, 1.0], 1],
 [[-0.0, 0.2, -0.0, -0.3], 1],
 [[-0.0, 0.2, -0.0, -0.3], 1],
 [[-0.0, 0.2, -0.0, -0.3], 1],
 [[-0.0, 0.2, -0.0, -0.3], 1],
 [[-0.0, 0.2, -0.0, -0.3], 1],
 [[-0.0, 0.4, -0.0, -0.6], 1],
 [[-0.0, 0.6, -0.0, -0.9], 1],
 [[-0.0, 0.4, -0.0, -0.6], 0],
 [[-0.0, 0.2, -0.1, -0.4], 0],
 [[-0.0, 0.0, -0.1, -0.1], 0],
 [[-0.0, -0.2, -0.1, 0.2], 0],
 [[-0.0, -0.4, -0.1, 0.5], 0],
 [[-0.0, -0.2, -0.0, 0.2], 1],
 [[-0.0, 0.0, -0.0, -0.1], 1],
 [[-0.0, 0.2, -0.0, -0.5], 1],
 [[-0.0, 0.4, -0.1, -0.8], 1],
 [[-0.0, 0.2, -0.1, -0.5], 0],
 [[-0.0, 0.0, -0.1, -0.2], 0],
 [[-0.0, 0.2, -0.1, -0.5], 1],
 [[

<h1 style="background-color: #9cffb6; text-align:center">Test Model (5 Frames)</h1>

now let's test it. first we need a function to get the action we need to perform.

In [333]:
def get_action(state):
    state_but_array = [round(state[0], DECIMALS), round(state[1], DECIMALS), round(state[2], DECIMALS), round(state[3], DECIMALS)]
    for entry in brain_array:
        if entry[0] == state_but_array:
            print('found state')
            return entry[1]
    return random.choice([0, 1])

now a function to test it

In [244]:
def test_and_train_model(attempts, max_score, visuals='on'):
    
    if visuals == 'off':
        env = gym.make("CartPole-v1")
    elif visuals == 'on':
        env = gym.make("CartPole-v1", render_mode="human") # uncomment this to actually see the progress (slows down the process terribly)

    # empty history each game
    state_history = []
    action_history = []
    # reset score
    score_array = []
    
    for episode in range(attempts):
        
        # reset all previous made progress in the environment
        env.reset()
        score=0
        
        for frame in range(max_score):
            
            # make the environment visible
            env.render()
        
            action = get_action(env.state)
            
            # results of the action
            _, _, game_over, _, _ = env.step(action)
            
            # the good stuff
            add_state(state_history, env.state, REMEMBER_AMOUNT_OF_FRAMES)
            add_action(action_history, action, REMEMBER_AMOUNT_OF_FRAMES)
            add_brain_cell_(state_history[0], action_history[0])
            
            # print results
            score+=1
            if game_over:
                score_array.append(score)
                break
        print('score: ' + str(score))
    print('average score: ' + str(get_avg_score(score_array, attempts)))

In [197]:
REMEMBER_AMOUNT_OF_FRAMES = 5
test_and_train_model(10, 500)

found state
found state
found state
found state
found state
found state
found state
found state
found state
score: 9
found state
found state
found state
found state
found state
found state
found state
found state
found state
score: 9
found state
found state
found state
found state
found state
found state
found state
found state
found state
score: 9
found state
found state
found state
found state
found state
found state
found state
found state
found state
found state
score: 10
found state
found state
found state
found state
found state
found state
found state
found state
found state
score: 9
found state
found state
found state
found state
found state
found state
found state
found state
found state
found state
score: 10
found state
found state
found state
found state
found state
found state
found state
found state
found state
found state
score: 10
found state
found state
found state
found state
found state
found state
found state
found state
found state
found state
found state
found stat

Ok so, it doesn't work at all, my conclusion is that we have to look more frames into the future or less, i think less since what might happen now is. It looks so far into the future that the moves that are saved have nothing to do with the current state anymore. Luckily I build my code so that we barely have to change anything. Let's go for 2 steps into future.

<h1 style="background-color: #9cffb6; text-align:center">Train Model (2 frames)</h1>

but first reset all data we have

In [239]:
brain_array = [[[2.0,2.0,2.0,2.0], 0]]

simply put a 2 here and it's all fixed

In [1015]:
REMEMBER_AMOUNT_OF_FRAMES = 2
train_model(1000, 100)

[1, 0]
score: 25
[1, 1]
score: 14
[0, 0]
score: 24
[1, 1]
score: 36
[1, 0]
score: 41
[1, 1]
score: 13
[0, 1]
score: 17
[1, 0]
score: 10
[0, 0]
score: 26
[1, 1]
score: 28
[0, 1]
score: 49
[0, 0]
score: 12
[0, 0]
score: 28
[1, 0]
score: 17
[1, 0]
score: 20
[1, 1]
score: 13
[1, 1]
score: 10
[1, 0]
score: 19
[1, 0]
score: 13
[0, 1]
score: 13
[1, 1]
score: 21
[1, 0]
score: 21
[0, 1]
score: 12
[1, 1]
score: 21
[1, 1]
score: 27
[1, 0]
score: 12
[1, 1]
score: 34
[1, 0]
score: 10
[1, 1]
score: 25
[1, 1]
score: 11
[1, 1]
score: 13
[0, 1]
score: 33
[0, 1]
score: 14
[0, 0]
score: 30
[1, 1]
score: 11
[1, 1]
score: 11
[1, 1]
score: 15
[1, 1]
score: 28
[0, 1]
score: 47
[1, 0]
score: 22
[0, 0]
score: 16
[1, 1]
score: 15
[0, 0]
score: 16
[0, 0]
score: 23
[0, 0]
score: 17
[0, 1]
score: 20
[1, 1]
score: 28
[1, 0]
score: 15
[1, 0]
score: 16
[0, 1]
score: 16
[0, 1]
score: 18
[0, 0]
score: 32
[0, 1]
score: 21
[1, 0]
score: 18
[1, 0]
score: 14
[0, 0]
score: 47
[1, 1]
score: 10
[0, 0]
score: 22
[0, 1]
score: 

[0, 0]
score: 17
[1, 0]
score: 18
[0, 1]
score: 14
[1, 0]
score: 20
[0, 1]
score: 19
[0, 1]
score: 29
[0, 0]
score: 27
[1, 0]
score: 24
[0, 0]
score: 26
[0, 1]
score: 50
[1, 0]
score: 28
[0, 1]
score: 14
[0, 0]
score: 32
[0, 1]
score: 16
[1, 0]
score: 31
[1, 1]
score: 12
[0, 1]
score: 15
[1, 1]
score: 15
[1, 0]
score: 19
[1, 1]
score: 11
[0, 0]
score: 22
[1, 0]
score: 14
[0, 1]
score: 15
[1, 0]
score: 13
[0, 1]
score: 15
[0, 1]
score: 18
[1, 0]
score: 52
[1, 1]
score: 18
[1, 1]
score: 12
[0, 1]
score: 12
[0, 0]
score: 29
[0, 1]
score: 13
[1, 1]
score: 29
[0, 0]
score: 13
[0, 1]
score: 18
[0, 1]
score: 16
[0, 1]
score: 17
[0, 1]
score: 31
[0, 0]
score: 17
[0, 1]
score: 19
[0, 0]
score: 18
[1, 1]
score: 16
[0, 1]
score: 29
[0, 1]
score: 11
[1, 1]
score: 15
[1, 1]
score: 20
[1, 0]
score: 9
[0, 1]
score: 29
[1, 0]
score: 18
[0, 1]
score: 19
[1, 1]
score: 18
[1, 0]
score: 21
[1, 0]
score: 33
[0, 1]
score: 19
[1, 1]
score: 29
[0, 0]
score: 12
[0, 1]
score: 20
[1, 0]
score: 46
[0, 0]
score: 1

<h1 style="background-color: #9cffb6; text-align:center">Test Model (2 Frames)</h1>

In [1021]:
test_model(1, 4, visuals='off')

score: 4
average score: 0.0


This makes no sense to me, I'm getting bad scores again. I should be doing something wrong right? 

to make sure my model is faulty, I tested it for 9 different amount of frame-methods. all of them gave horrible scores. Legit every frame it encounters during the test has been encountered before but still it makes bad moves. That is very weird.

<table>
  <tr>
    <th>Frames looked ahead</th>
    <th>Amount of frames trained</th>
    <th>average score (100 attempts)</th>
  </tr>
  <tr>
    <td>1</td>
    <td>23900</td>
    <td>9.83</td>
  </tr>
  <tr>
    <td>2</td>
    <td>23332</td>
    <td>10.08</td>
  </tr>
  <tr>
    <td>3</td>
    <td>23335</td>
    <td>11.58</td>
  </tr>
  <tr>
    <td>4</td>
    <td>23766</td>
    <td>11.44</td>
  </tr>
  <tr>
    <td>5</td>
    <td>22804</td>
    <td>9.6</td>
  </tr>
  <tr>
    <td>6</td>
    <td>23533</td>
    <td>10.95</td>
  </tr>
  <tr>
    <td>7</td>
    <td>24025</td>
    <td>12.64</td>
  </tr>
  <tr>
    <td>8</td>
    <td>23554</td>
    <td>10.22</td>
  </tr>
  <tr>
    <td>9</td>
    <td>24102</td>
    <td>10.78</td>
  </tr>
</table>

let's try to train and test at the same time, maybe that will change anything.

we need to modify our `test_model()` function a bit to make it train and test at the same time

In [291]:
brain_array = [[[2.0,2.0,2.0,2.0], 0]]

In [292]:
REMEMBER_AMOUNT_OF_FRAMES = 3
test_and_train_model(1, 10)

[0.0, 0.2, -0.0, -0.3]
[0.0, 0.2, -0.0, -0.3]
found state
score: 10
average score: 10.0


In [284]:
brain_array

[[[2.0, 2.0, 2.0, 2.0], 0],
 [[-0.0, 0.2, 0.0, -0.3], 1],
 [[-0.0, 0.2, 0.0, -0.3], 1]]

<h1 style="background-color: #9cffb6; text-align:center">Troubleshooting</h1>

## I found problems
I can confirm something is going wrong. 
### Problem 1
In the first frame of the game we start saving data. That shouldn't be happening. We should start saving data after x amount of frames have passed. 
### Problem 2
About the other problem, it always says that it has seen this state before. even without training
### Problem 3
we save the same states multiple times in our data
### Problem 4 
we round of to 1 decimal, but one of the most important variables is almost always 0.0 because of this, so lets round that variable off to 1 more decimal decimals.

Fixing problem 3 and 4

In [501]:
DECIMALS = 1

# fixing problem 3
def add_brain_cell(state, action):
    for entry in brain_array:
        state_but_array = [round(state[0], DECIMALS), round(state[1], DECIMALS), round(state[2], DECIMALS), round(state[3], DECIMALS)]
        if entry[0] == state_but_array:
            #print('same state')
            return True
    brain_array.append([state_but_array, action])
                
            
            

Fixing problem 1 and some other problems i found later on

In [996]:
def get_action(state):
    state_but_array = [round(state[0], DECIMALS), round(state[1], DECIMALS), round(state[2], DECIMALS), round(state[3], DECIMALS)]
    for entry in brain_array:
        if entry[0] == state_but_array:
            #print('same')
            return entry[1]
    #print('new')
    #return random.choice([0, 1])
    return 0

In [650]:
REMEMBER_AMOUNT_OF_FRAMES = 5

def train_model_2(attempts, max_score, visuals='off'):
    
    if visuals == 'off':
        env = gym.make("CartPole-v1")
    else:
        env = gym.make("CartPole-v1", render_mode="human") # uncomment this to actually see the progress (slows down the process terribly)

    
    # empty history each game
    state_history = []
    action_history = []
    score_array = []
    
    for episode in range(attempts):
        
        # reset all previous made progress in the environment
        env.reset()
        score=0
        
        for frame in range(max_score):
            
            # make the environment visible
            env.render()
            
            # generate random action (0=left or 1=right)
            action = get_action(env.state)
            #print('frame: ' + str(frame))
            #action = 0
            #print(env.state)
            
            # output of the action
            _, _, game_over, _, _ = env.step(action)
            
            # fixing problem 1
            add_state(state_history, env.state, REMEMBER_AMOUNT_OF_FRAMES)
            add_action(action_history, action, REMEMBER_AMOUNT_OF_FRAMES)
            if frame > REMEMBER_AMOUNT_OF_FRAMES:    
                add_brain_cell(state_history[0], action_history[0])
            
            # print results
            score+=1
            if game_over:
                score_array.append(score)
                break
        print('score: ' + str(score))
    print('average score: ' + str(get_avg_score(score_array, attempts)))

Let's see what happens

In [644]:
REMEMBER_AMOUNT_OF_FRAMES = 5
train_model_2(1000000000000, 100, visuals='off')

new
new
score: 15
new
new
new
score: 11
new
score: 10
new
score: 11
new
new
score: 16
new
new
score: 13
new
new
score: 9
new
score: 9
new
new
score: 15
new
new
score: 11
new
new
score: 9
new
new
score: 17
new
score: 25
new
new
score: 14
new
new
score: 9
new
new
score: 10
new
score: 8
new
new
score: 15
new
score: 10
new
score: 9
new
new
score: 15
new
score: 12
new
score: 14
new
score: 10
new
score: 10
new
score: 19
new
score: 19
new
score: 19
new
score: 14
new
new
score: 15
new
new
score: 15
new
score: 21
new
new
new
score: 11
new
score: 15
new
score: 9
new
score: 10
new
score: 19
new
score: 12
new
new
score: 17
new
score: 17
new
score: 15
new
new
score: 10
new
score: 8
new
score: 17
new
score: 21
new
score: 10
new
score: 17
new
new
new
score: 11
new
score: 17
new
score: 8
new
score: 15
new
score: 17
new
score: 15
new
new
score: 15
new
score: 15
new
new
score: 12
new
score: 18
new
score: 13
new
new
score: 13
new
score: 11
new
score: 15
new
score: 23
score: 16
new
score: 10
new
new
score

KeyboardInterrupt: 

Now when i train my model, it only dies if it encounters a new states. If it EVER encounters a state it has seen before, it knows what to do and survives. This is a piece of verification for myself that its finally working correctly. Let's see what we can do.

Something else i noticed is,

if you look less frames into the future, you need less new states to die. So you would say, the more frames you look into the future, the better your surviving chances are. Correct, that's true, but. the more frames it looks into the future, the slower it trains.

This is because for example you train looking 10 frames into the future. that means you need to at least survive 11 frames. if you do random inputs you get an average score of 20. this means you learn 9 new states on average when you play. if you reduce this to looking 2 frames into the future you learn an average of 18 frames but you are likely to die even though you are trained, because looking 2 frames into the future just isnt enough. The good spot is somewhere around 6/7. Right now I am training a model that looks 7 frames into the future, curious what it can do.

7 trains hella slow lets do 6 

in 1000 test games we only gain like 8 new states entries. That's actually horrible let's try 5

I can keep lowering the frame number untill it dies while it recognizes the last x states. If that ever happens we know we should raise it.

In [657]:
REMEMBER_AMOUNT_OF_FRAMES = 3
train_model_2(10, 100, visuals='off')

score: 8
new
score: 17
new
score: 12
new
score: 17
new
score: 15
new
score: 17
new
score: 15
new
score: 10
score: 11
new
score: 17
average score: 13.9


with 3 frames looking ahead you can see it already go wrong, you can find games played where we die without encountering new states. this means 3 frames is just not enough, lets try to look 4 frames in to the future.

In [656]:
REMEMBER_AMOUNT_OF_FRAMES = 4
train_model_2(10, 100, visuals='off')

score: 18
new
score: 16
new
score: 18
new
score: 14
new
score: 12
new
score: 12
new
new
score: 18
score: 18
new
score: 20
score: 10
average score: 15.6


4 had same issue lets try 5

In [658]:
REMEMBER_AMOUNT_OF_FRAMES = 5
train_model_2(10, 100, visuals='off')

new
score: 8
new
score: 15
new
score: 15
score: 12
new
score: 10
new
score: 15
new
score: 10
new
new
score: 38
new
score: 11
new
score: 17
average score: 15.1


5 has the same issue damn, means we need to go to 6, but 6 trains so slow.

In [676]:
REMEMBER_AMOUNT_OF_FRAMES = 6
train_model_2(1, 100, visuals='on')

new
new
new
score: 18
average score: 18.0


just wanted to add this here, 6 frame model learns 33 new states in 10000 games. awesome 🤠👍

In [678]:
save_model('1dec-6frame-model-v1', brain_array)

In [680]:
len(brain_array)

1754

# Not really a problem but...
So we are kinda having a problem, we can train, sure. but this goes so insanely slow. Because there are so many states we can encounter. HOWEVER, we can speed this up. What if I would tell you we could do something right now to receive all **MISSING DATA**. Are you getting goosebumps right now? Well I am. Sadly I have never fixed any missing data problem with an actual-data-dataset. Sucks 🤠👍

If I knew how missing data worked I could maybe write a script that would automatically fill in missing data, that would be peak coding, but sadly.

# 2 ways to fix it
I came up with 2 ideas to get better scores from this point

### method a - Handle Missing Data (MAR)?
i feel like my data has a structure to it, so it should be missing at random data.

### method b - Combine all methods
so previously we got quite good scores taking into account pole angles and pole speed. So what I was thinking is, what if we use that code, and overwrite any state where it dies. 

### method c - I AM SO STUPID
ok, i am really stupid, the way to fix this is so easy. the last few moves that kill us every time. we don't learn anything from it, but we could learn something from it. Right now we just throw away the last 6 states and last 6 actions before we die. What we could do is try to invert the last 6 actions and save it as data. sounds good right? lets try that

instead of showing all code, let me just spoil it, it didnt work. I just wasted 45 minutes on crap

# THE MOMENT WE'VE ALL BEEN WAITING FOR

<h1 style="background-color: #9cffb6; text-align:center">My Model vs Random Agent (Rematch)</h1>

In [992]:
def random_agent(attempts, max_score, visuals='off'):
    env = gym.make("CartPole-v1")
    score_array = []
    for episode in range(attempts):
        env.reset()
        score=0
        for frame in range(max_score):
            env.render()
            action = random.choice([0, 1])
            _, _, game_over, _, _ = env.step(action)
            score+=1
            if game_over:
                score_array.append(score)
                break
        print('score: ' + str(score))
    print('average score: ' + str(get_avg_score(score_array, attempts)))

In [993]:
random_agent(100, 500)

score: 13
score: 15
score: 16
score: 13
score: 19
score: 8
score: 12
score: 35
score: 31
score: 14
score: 13
score: 27
score: 32
score: 14
score: 30
score: 18
score: 30
score: 14
score: 36
score: 45
score: 22
score: 22
score: 27
score: 12
score: 59
score: 11
score: 41
score: 29
score: 15
score: 13
score: 15
score: 22
score: 11
score: 20
score: 9
score: 22
score: 19
score: 23
score: 21
score: 29
score: 15
score: 21
score: 35
score: 29
score: 47
score: 26
score: 10
score: 14
score: 18
score: 11
score: 11
score: 18
score: 42
score: 20
score: 10
score: 29
score: 10
score: 11
score: 8
score: 24
score: 34
score: 99
score: 10
score: 33
score: 42
score: 12
score: 19
score: 11
score: 37
score: 22
score: 18
score: 14
score: 46
score: 16
score: 44
score: 29
score: 57
score: 15
score: 27
score: 19
score: 31
score: 19
score: 10
score: 19
score: 63
score: 25
score: 24
score: 84
score: 23
score: 13
score: 19
score: 16
score: 12
score: 23
score: 24
score: 14
score: 10
score: 12
score: 18
score: 15
ave

Bot kicks it off with a nice 23.64

In [ ]:
Here is the total scoreboard (i also uncluded scores from the other document)

<table>
  <tr>
    <th>Rank</th> 
    <th>name</th>
    <th>Method</th>
    <th>Amount of frames trained</th>
    <th>average score (100 attempts)</th>
  </tr>
  <tr>
    <td>1</td>
    <td>1dec-model-both-v1</td>
    <td>Pole Angle + Pole Speed</td>
    <td>121466</td>
    <td>136.17</td>
  </tr>
  <tr>
    <td>2</td>
    <td>1dec-model-angle-v1</td>
    <td>Pole Angle</td>
    <td>131121</td>
    <td>107.00</td>
  </tr>
  <tr>
    <td>3</td>
    <td>1dec-model-v3</td>
    <td>Pole Speed</td>
    <td>11846</td>
    <td>99.89</td>
  </tr>
  <tr>
    <td>4</td>
    <td>Random Agent (2)</td>
    <td>Random</td>
    <td>0</td>
    <td>23.64</td>
  </tr>
  <tr>
    <td>5</td>
    <td>Random Agent (1)</td>
    <td>Random</td>
    <td>0</td>
    <td>23.4</td>
  </tr>
  <tr>
    <td>6</td>
    <td>1dec-model-v2</td>
    <td>Pole Speed</td>
    <td>3050</td>
    <td>19.08</td>
  </tr>
</table>

Now it's my turn

In [1008]:
REMEMBER_AMOUNT_OF_FRAMES = 5
brain_array = load_model('1dec-5frame-model-v1')
train_model_2(100, 500)

score: 9
score: 18
score: 10
score: 8
score: 14
score: 14
score: 10
score: 11
score: 11
score: 10
score: 15
score: 11
score: 9
score: 15
score: 20
score: 15
score: 11
score: 10
score: 12
score: 13
score: 15
score: 14
score: 8
score: 19
score: 12
score: 13
score: 19
score: 17
score: 19
score: 10
score: 12
score: 8
score: 17
score: 19
score: 10
score: 16
score: 11
score: 12
score: 15
score: 10
score: 21
score: 10
score: 10
score: 10
score: 15
score: 17
score: 13
score: 17
score: 15
score: 13
score: 14
score: 9
score: 17
score: 10
score: 10
score: 19
score: 15
score: 14
score: 10
score: 21
score: 21
score: 10
score: 11
score: 14
score: 10
score: 16
score: 12
score: 10
score: 8
score: 11
score: 8
score: 15
score: 10
score: 11
score: 15
score: 21
score: 15
score: 15
score: 9
score: 12
score: 21
score: 12
score: 15
score: 10
score: 10
score: 15
score: 20
score: 9
score: 17
score: 19
score: 17
score: 19
score: 23
score: 15
score: 12
score: 22
score: 39
score: 12
score: 15
score: 13
average sc

In [1009]:
REMEMBER_AMOUNT_OF_FRAMES = 6
brain_array = load_model('1dec-6frame-model-v1')
train_model_2(100, 500)

score: 10
score: 10
score: 10
score: 11
score: 11
score: 9
score: 9
score: 9
score: 15
score: 10
score: 11
score: 10
score: 13
score: 9
score: 9
score: 10
score: 14
score: 9
score: 16
score: 18
score: 18
score: 18
score: 10
score: 9
score: 10
score: 18
score: 10
score: 13
score: 9
score: 10
score: 24
score: 10
score: 9
score: 14
score: 9
score: 14
score: 10
score: 11
score: 18
score: 15
score: 14
score: 11
score: 10
score: 10
score: 11
score: 20
score: 18
score: 9
score: 10
score: 14
score: 18
score: 11
score: 10
score: 10
score: 11
score: 10
score: 10
score: 15
score: 9
score: 13
score: 9
score: 15
score: 13
score: 10
score: 10
score: 18
score: 10
score: 10
score: 21
score: 9
score: 9
score: 10
score: 15
score: 11
score: 10
score: 11
score: 11
score: 9
score: 10
score: 9
score: 10
score: 9
score: 18
score: 10
score: 10
score: 21
score: 10
score: 10
score: 9
score: 9
score: 11
score: 15
score: 10
score: 11
score: 13
score: 13
score: 10
score: 11
score: 15
score: 14
average score: 11.95

Both my models scored horrible, like actually horrible. But the table looks gorgeous, having all these variables in a table really sums up a fun story.

<table>
  <tr>
    <th>Rank</th> 
    <th>name</th>
    <th>Method</th>
    <th>Amount of frames trained</th>
    <th>average score (100 attempts)</th>
  </tr>
  <tr>
    <td>1</td>
    <td>1dec-model-both-v1</td>
    <td>Pole Angle + Pole Speed</td>
    <td>121466</td>
    <td>136.17</td>
  </tr>
  <tr>
    <td>2</td>
    <td>1dec-model-angle-v1</td>
    <td>Pole Angle</td>
    <td>131121</td>
    <td>107.00</td>
  </tr>
  <tr>
    <td>3</td>
    <td>1dec-model-v3</td>
    <td>Pole Speed</td>
    <td>11846</td>
    <td>99.89</td>
  </tr>
  <tr>
    <td>4</td>
    <td>Random Agent (2)</td>
    <td>Random</td>
    <td>0</td>
    <td>23.64</td>
  </tr>
  <tr>
    <td>5</td>
    <td>Random Agent (1)</td>
    <td>Random</td>
    <td>0</td>
    <td>23.4</td>
  </tr>
  <tr>
    <td>6</td>
    <td>1dec-model-v2</td>
    <td>Pole Speed</td>
    <td>3050</td>
    <td>19.08</td>
  </tr>
  <tr>
    <td>7</td>
    <td>1dec-5frame-model-v1</td>
    <td>Look 5 frames ahead</td>
    <td>1072</td>
    <td>13.91</td>
  </tr>
  <tr>
    <td>8</td>
    <td>1dec-6frame-model-v1</td>
    <td>Look 6 frames ahead</td>
    <td>1754</td>
    <td>11.95</td>
  </tr>
</table>

<h1 style="background-color: #9cffb6; text-align:center">Conclusion</h1>

we can conclude that the more a model is trained, the better it performs. Are the pole methods better than the frame methods? in the short term yes, but if the frame models get trained as much as the pole models they will definitely crush the pole models. the problem is that they train very slow (in 10000 games they learn 33 new states). This is my bad because the reward system on the frame method is just bad. So they idea of the frame method is very neat, but poorly executed. Instead of trying to improve I will leave it off here. It was a very cool adventure, and definitely learned lots of cool things.

we haven't hit 500 points 😿